# Tutorial-2

This third exercise demonstrates how to achieve basic interoperability between SEG-Y and HDF5eis files. To begin, we need to install the `segyio` package

In [ ]:
!pip install segyio

Now let's import some necessary packages for this tutorial and create a directory where we can write some data files

In [ ]:
# Standard library imports
import io
import pathlib
import tempfile
import urllib

# Third-party imports
import hdf5eis
import numpy as np
import obspy.clients.fdsn
import pandas as pd
import segyio


tmp_dir = tempfile.TemporaryDirectory()
OUTPUT_DIR = pathlib.Path(tmp_dir.name)
# OUTPUT_DIR.mkdir(exist_ok=True, parents/=True)

Download a few SEG-Y example files from https://pubs.usgs.gov/ds/652/segy. This may take a few minutes to complete.

In [ ]:
segy_dir = OUTPUT_DIR.joinpath('segy')
segy_dir.mkdir(exist_ok=True, parents=True)

for name in ('sb6a_sgy.tra', 'sb_b10a.tra', 'sb_b10b.tra'):
    url = f'https://pubs.usgs.gov/ds/652/segy/96fgs01/{name}'
    path = segy_dir.joinpath(name)
    print(f'Downloading {url} to {path}.')
    urllib.request.urlretrieve(url, filename=path)

Define a few functions to extract metadata and trace data from SEG-Y files.

In [ ]:
def extract_header(path):
    with segyio.open(path, strict=False, ignore_geometry=True) as in_file:
        header = pd.DataFrame([
            {
                str(field): in_file.header[iheader][field]
                for field in segyio.TraceField.enums()
                if field in in_file.header[iheader]
            }
            for iheader in range(len(in_file.header))
        ])
    return header

def extract_binary_header(path):
    with segyio.open(path, strict=False, ignore_geometry=True) as in_file:
        header = pd.DataFrame([
            {
                str(field): in_file.bin[field]
                for field in segyio.BinField.enums()
                if field in in_file.bin
            }
        ])
    return header

def extract_trace_data(path):
    with segyio.open(path, strict=False, ignore_geometry=True) as in_file:
        trace_data = np.stack([
            in_file.trace[itrace] for itrace in range(len(in_file.trace))
        ])
    return trace_data

Extract data for a sample SEG-Y file.

In [ ]:
path = list(segy_dir.iterdir())[0]

header = extract_header(path)
binary_header = extract_binary_header(path)
trace_data = extract_trace_data(path)

Display the contents of each of these.

In [ ]:
header

In [ ]:
binary_header

In [ ]:
plt.close('all')
fig, ax = plt.subplots()
ax.pcolorfast(trace_data.T)
ax.invert_yaxis()

Iterate over SEG-Y files and write data to HDF5eis format.

In [ ]:
hdf5eis_path = OUTPUT_DIR.joinpath('test.hdf5')
with hdf5eis.File(hdf5eis_path, mode='w', overwrite=True) as out_file:
    for path in sorted(segy_dir.glob('*.tra')):
        tag = path.name.removesuffix('.tra')
        print(f'Saving data from {path} under tag {tag}.')
        out_file.metadata.add_table(extract_header(path), f'{tag}/header')
        out_file.metadata.add_table(extract_binary_header(path), f'{tag}/binary_header')
        out_file.metadata.add_utf8(str(path), f'{tag}/path')
        # Convert sample interval in microseconds to sampling rate in seconds.
        sampling_rate = 1/(np.mean(header['TRACE_SAMPLE_INTERVAL'])*1e-6)
        out_file.timeseries.add(
            extract_trace_data(path),
            0,   # Set the start time to 0.
            sampling_rate,
            tag
        )

Now we can easily retrieve trace and header data for a given tag.

In [ ]:
tag = 'sb6a_sgy'
hdf5eis_path = OUTPUT_DIR.joinpath('test.hdf5')

with hdf5eis.File(hdf5eis_path, mode='r', validate=False) as in_file:
    # Retrieve one second of timeseries data.
    super_gather = in_file.timeseries[
        tag,    # We retrieve data for a single tag.
        ...,    # Across all channels.
        0: 1e9 # The desired time range is specified in nanoseconds.
    ]
    header = in_file.metadata[f'{tag}/header']

In [ ]:
plt.close('all')
fig, ax = plt.subplots()
ax.pcolorfast(super_gather[tag][0].data.T)
ax.invert_yaxis()

And we can convert data back to SEG-Y format.

In [ ]:
def to_dict(row, module):
    '''
    Convert a header row to a dict.
    '''
    return dict(zip(
        [getattr(module, field) for field in row.index],
        row
    ))

In [ ]:
# Open the HDF5eis file in read mode.
with hdf5eis.File(hdf5eis_path, mode='r', validate=False) as in_file:
    # Loop over unique tags in data set.
    for tag in in_file.timeseries.index['tag'].unique():
        index = in_file.timeseries.index
        index = index.set_index('tag')
        index = index.loc[[tag]]
        start_time = index['start_time'].min()
        end_time   = index['end_time'].max()
        # Create a new output file path.
        path = segy_dir.joinpath(f'{tag}-new.sgy')
        print(f'Converting tag {tag} back to SEG-Y format: {path}.')
        # Read the data set associated with tag from HDF5eis.
        super_gather = in_file.timeseries[tag, ..., start_time: end_time]
        # Extract the data array
        trace_data = super_gather[tag][0].data
        # Get the corresponding headers
        binary_header, _ = in_file.metadata[f'{tag}/binary_header']
        header, _ = in_file.metadata[f'{tag}/header']
        # Write the trace data to SEG-Y format.
        segyio.tools.from_array2D(
            path, 
            trace_data, 
            format=binary_header['Format']
        )

        # Update the SEG-Y headers.
        with segyio.open(path, mode='r+', ignore_geometry=True) as out_file:
            out_file.bin = to_dict(binary_header.iloc[0], segyio.BinField)
            for i, header_row in header.iterrows():
                out_file.header[i] = to_dict(header_row, segyio.TraceField)